# Install Taipy

To install Taipy, just `pip install` it.

In [1]:
pip install taipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pmdarima

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


# Import the packages et initialization

In [4]:
from taipy.gui import Gui, Markdown, notify
from taipy import Config
import taipy as tp

import datetime as dt

from pmdarima import auto_arima

import pandas as pd
import numpy as np

import requests

In [5]:
from pandas.core.common import SettingWithCopyWarning
import warnings

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

data_country_date = None
selected_country = None

selected_scenario = None
scenario_selector = None

scenario_name = None

result = None

selected_points = []

data_province_displayed = None

api_year = None
api_pop = None
api_country = None

# Taipy Gui Basics
## Markdown Syntax

Taipy uses the Markdown syntax to display elements. `#` creates a title, `*` puts your text in italics and `**` puts it in bold.

![](img/gui_basic_eng.png)

In [6]:
page_md = """
# Taipy

Test **here** to put some *markdown*

Click to access the [doc](https://docs.taipy.io/en/latest/)
"""

In [7]:
page = Markdown(page_md)

gui = Gui(page=page)
gui.run(dark_mode=False)

 * Server starting on http://127.0.0.1:5000


## Visual elements
Create different visual elements. The syntax is always the same for each visual element.  `<|{value}|name_of_visual_element|property_1=value_of_property_1|...|>`
- Create a [slider](https://docs.taipy.io/en/latest/manuals/gui/viselements/slider/) `<|{value}|slider|>`

- Create a [date](https://docs.taipy.io/en/latest/manuals/gui/viselements/date/) `<|{value}|date|>`

- Create a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) `<|{value}|selector|lov={list_of_values}|>`


![](img/control.png)

In [52]:
slider_value = 0
date_value = '2022-02-03'
selected_value = None

In [53]:
selector = ['Test 1', 'Test 2', 'Test 3']

control_md = """
## Controls

<|{slider_value}|slider|> <|{slider_value}|>

<|...|> <|{date_value}|>

<|...|>  <|{selected_value}|>
"""

In [54]:
gui.stop()
page.set_content(control_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:5000


The WebSocket transport is not available, you must install a WebSocket server that is compatible with your async mode to enable it. See the documentation for details. (further occurrences of this error will be logged with level INFO)


## Data Viz

A dataset gathering information on the number of deaths, confirmed cases and recovered for different regions is going to be used to create an interactive Dashboard.

In [11]:
path_to_data = "data/covid-19-all.csv"
data = pd.read_csv(path_to_data, low_memory=False)
data[-5:]

The WebSocket transport is not available, you must install a WebSocket server that is compatible with your async mode to enable it. See the documentation for details. (further occurrences of this error will be logged with level INFO)


,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
1241947,Vietnam,NaN,14.058324,108.277199,1465.0,1325.0,35.0,2020-12-31
1241948,West Bank and Gaza,NaN,31.952200,35.233200,138004.0,117183.0,1400.0,2020-12-31
1241949,Yemen,NaN,15.552727,48.516388,2099.0,1394.0,610.0,2020-12-31
1241950,Zambia,NaN,-13.133897,27.849332,20725.0,18660.0,388.0,2020-12-31
1241951,Zimbabwe,NaN,-19.015438,29.154857,13867.0,11250.0,363.0,2020-12-31


In [12]:
def initialize_case_evolution(data, selected_country='France'):
    # Aggregation of the dataframe per Country/Region
    data_country_date = data.groupby(["Country/Region",'Date'])\
                            .sum()\
                            .reset_index()
    
    # a country is selected, here France by default
    data_country_date = data_country_date.loc[data_country_date['Country/Region']==selected_country]
    return data_country_date

In [13]:
data_country_date = initialize_case_evolution(data)
data_country_date.head()

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
18101,France,2020-01-24,14.6415,-56.3159,2.0,0.0,0.0
18102,France,2020-01-25,14.6415,-56.3159,3.0,0.0,0.0
18103,France,2020-01-26,14.6415,-56.3159,3.0,0.0,0.0
18104,France,2020-01-27,14.6415,-56.3159,3.0,0.0,0.0
18105,France,2020-01-28,14.6415,-56.3159,4.0,0.0,0.0


Create a [chart](https://docs.taipy.io/en/latest/manuals/gui/viselements/chart/) showing the evolution of Deaths in France (_Deaths_ for _y_ and _Date_ for _x_). The visual element (chart) has the same syntax as the other ones with specific properties (_x_, _y_, _type_ for example). Here are some [examples of charts](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/charts/bar/). The _x_ and _y_ porperties only needs the name of the dataframe columns to display.

![](img/simple_graph.png)

In [14]:
country_md = "(-> create a chart showing the evolution of Deaths <-)"

In [15]:
gui.stop()
page.set_content(country_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:5000


## Add new traces

- Add on the graph the number of Confirmed and Recovered cases (_Confirmed_ and _Recovered_) with the number of Deaths
- _y_ (and _x_) can be indexed this way to add more traces (`y[1]=`, `y[2]=`, `y[3]=`).

![](img/multi_traces.png)

In [16]:
country_md = "<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|...|>"

In [17]:
gui.stop()
page.set_content(country_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:5000


## Style the graph with personnalized properties
The _layout_ dictionnary specifies how bars should be displayed. They would be 'stacked'.

The _options_ dictionary will change the opacity of the unselected markers.

These are Plotly properties.

![](img/stack_chart.png)

In [18]:
layout = {'barmode':'stack'}
options = {"unselected":{"marker":{"opacity":0.5}}}
country_md = "<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>"

In [19]:
gui.stop()
page.set_content(country_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:5000


## Add texts that sums up the data

Use the [text](https://docs.taipy.io/en/latest/manuals/gui/viselements/text/) visual element.

- Add the total number of Deaths (last line of _data_country_date_)
- Add the total number of Recovered (last line of _data_country_date_)
- Add the total number of Confirmed (last line of _data_country_date_)


In [20]:
data_country_date

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
18101,France,2020-01-24,14.641500,-56.315900,2.0,0.0,0.0
18102,France,2020-01-25,14.641500,-56.315900,3.0,0.0,0.0
18103,France,2020-01-26,14.641500,-56.315900,3.0,0.0,0.0
18104,France,2020-01-27,14.641500,-56.315900,3.0,0.0,0.0
18105,France,2020-01-28,14.641500,-56.315900,4.0,0.0,0.0
...,...,...,...,...,...,...,...
18439,France,2020-12-27,91.463495,-238.646914,2616510.0,195861.0,62867.0
18440,France,2020-12-28,91.463495,-238.646914,2619616.0,196642.0,63235.0
18441,France,2020-12-29,91.463495,-238.646914,2631110.0,197726.0,64204.0
18442,France,2020-12-30,91.463495,-238.646914,2657624.0,198966.0,64508.0


This is how we can get the total number of Deaths from the daatset for France.

In [21]:
data_country_date.iloc[-1, 6] # gives the number of deaths for France (5 is for recovered and 4 is confirmed)

64759.0

Use the [text](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/text/) visual element. Note that between `{}`, any Python variable can be put but also any Python code.

![](img/control_text.png)

In [22]:
country_md = """
## Deaths <|{data_country_date.iloc[-1, 6]}|text|>

## Recovered <|...|text|>

## Confirmed <|...|text|>

<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [23]:
gui.stop()
page.set_content(country_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:5000


## Local _on_change_

- Add a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) with `dropdown=True` containing the name of all the _Country/region_
- Give to the _on_change_ selector property the name of the _on_change_country_ function. This function will be called when the selector will be used.
- This function has a 'state' parameter and has to be completed. When the selector is used, this function is called with the _state_ argument. It contains all the Gui variables; 'state.data_country_date' is then the dataframe used in the Gui.

![](img/on_change_local.png)

In [24]:
selector_country = list(data['Country/Region'].astype(str).unique())
selected_country = 'France'

country_md = """
(-->create a selector with:
        selected_country as a selection,
        selector_country as a lov,
        and with on_change=on_change_country in the properties <--)

## Deaths <|{data_country_date.iloc[-1, 6]}|>

## Recovered <|{data_country_date.iloc[-1, 5]}|>

## Confirmed <|{data_country_date.iloc[-1, 4]}|>

<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [25]:
def on_change_country(state):
    # state contains all the Gui variables and this is through this state variable that we can update the Gui
    # state.selected_country, state.data_country_date, ...
    # update data_country_date with the right country (use initialize_case_evolution)
    print("Chosen country: ", state.selected_country)
    ...

In [26]:
gui.stop()
page.set_content(country_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:5000


## Layout

Use the [layout](https://docs.taipy.io/en/latest/manuals/gui/viselements/layout/) block to change the page structure. This block creates invisible columns to put text/visual elements in.

Syntax :
```
<|layout|columns=1 1 1 ...|
(firsrt column)

(in second column)

(third column)
(again, third column)

(...)
|>
```

In [27]:
final_country_md = """
(-> beginning of layout block <-)
<|{selected_country}|selector|lov={selector_country}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{data_country_date.iloc[-1, 6]}|>

## Recovered <|{data_country_date.iloc[-1, 5]}|>

## Confirmed <|{data_country_date.iloc[-1, 4]}|>
(-> end of layout block <-)

<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [28]:
gui.stop()
page.set_content(final_country_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:5000


![](img/layout.png)

# Map

In [29]:
def initialize_map(data):
    data['Province/State'] = data['Province/State'].fillna(data["Country/Region"])
    data_province = data.groupby(["Country/Region",
                                  'Province/State',
                                  'Longitude',
                                  'Latitude'])\
                         .max()
                         

    data_province_displayed = data_province[data_province['Deaths']>10].reset_index()

    data_province_displayed['Size'] = np.sqrt(data_province_displayed.loc[:,'Deaths']/data_province_displayed.loc[:,'Deaths'].max())*80 + 3
    data_province_displayed['Text'] = data_province_displayed.loc[:,'Deaths'].astype(str) + ' deaths </br> ' + data_province_displayed.loc[:,'Province/State']
    return data_province_displayed

In [30]:
data_province_displayed = initialize_map(data)
data_province_displayed.head()

,Country/Region,Province/State,Longitude,Latitude,Confirmed,Recovered,Deaths,Date,Size,Text
0,Afghanistan,Afghanistan,67.709953,33.93911,51526.0,41727.0,2191.0,2020-12-31,17.771247,2191.0 deaths </br> Afghanistan
1,Albania,Albania,20.168300,41.15330,58316.0,33634.0,1181.0,2020-12-31,13.844784,1181.0 deaths </br> Albania
2,Algeria,Algeria,1.659600,28.03390,99610.0,67127.0,2756.0,2020-12-31,19.566684,2756.0 deaths </br> Algeria
3,Andorra,Andorra,1.521800,42.50630,8049.0,7432.0,84.0,2020-12-31,5.892249,84.0 deaths </br> Andorra
4,Angola,Angola,17.873900,-11.20270,17553.0,11044.0,405.0,2020-12-31,9.350728,405.0 deaths </br> Angola


Properties to style the map
- marker color corresponds to the number of Deaths (column _Deaths_)
- marker sizes corresponds to the size in _Size_ column which is created from the number of Deaths

layout_map permet defined the initial zoom and position of the map


In [31]:
marker_map = {"color":"Deaths", "size": "Size", "showscale":True, "colorscale":"Viridis"}
layout_map = {
            "dragmode": "zoom",
            "mapbox": { "style": "open-street-map", "center": { "lat": 38, "lon": -90 }, "zoom": 3}
            }

We give to Plotly:
- a map type
- the name of the latitude column
- the name of the longitude column
- properties: on the size and color of the markers
- the name of the column for the text of the points

In [32]:
map_md = """
<|{data_province_displayed}|chart|type=scattermapbox|selected={selected_points}|lat=Latitude|lon=Longitude|marker={marker_map}|layout={layout_map}|text=Text|mode=markers|height=800px|options={options}|>
"""

In [33]:
gui.stop()
page.set_content(map_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:5000


![](img/carte.png)

### API call when a point is selected

When a point is selected on the map, data will be actualized in the _expandable_ block through an API call.

To achieve this, we need the _selected_ property of the _chart_ and to handle its change through the global _on_change_ function.

In [34]:
map_md = """
<|Information|expandable|expanded=False|
<|layout|columns=1 1|
Country: <|{api_country}|>

Population: (Year: <|{api_year}|>, Value: <|{api_pop}|>)
|>
|>

<br/>
<|{data_province_displayed}|chart|type=scattermapbox|selected={selected_points}|lat=Latitude|lon=Longitude|marker={marker_map}|layout={layout_map}|text=Text|mode=markers|height=800px|options={options}|>
"""

In [35]:
url_pop = "https://countriesnow.space/api/v0.1/countries/population"

def api_call(state):
    state.api_country = state.data_province_displayed.loc[state.selected_points[0], "Country/Region"]
    
    if state.api_country == 'US':
        state.api_country = "United States"
    
    body = {"country": state.api_country}
    
    response_pop = requests.post(url_pop, body).json()
    
    if not response_pop['error']:
        state.api_year = response_pop['data']['populationCounts'][-1]['year']
        state.api_pop = response_pop['data']['populationCounts'][-1]['value']

In [36]:
def on_change(state, var_name, var_value):
    print(var_name, var_value)
    # on_change is called each time a Gui variable is changed (either through the Gui or with state.xxx = yyy
    # 1) call 'api_call' when selected_points changes
    ...

In [37]:
gui.stop()
gui.on_change = on_change
page.set_content(map_md)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:5000


# Part and the _render_ property
- Create a [toggle](https://docs.taipy.io/en/latest/manuals/gui/viselements/toggle/) (works the same as a selector) with a lov of 'Map' an 'Country'
- Create two part blocks that renders or not depending on the value of the toggle
    - To do this, use the fact that in the _render_ property of the part block, Python code can be inserted in `{}`

In [38]:
representation_selector = ['Map', 'Country']
selected_representation = representation_selector[0]

In [39]:
main_page = """
<|{selected_representation}|toggle|lov={representation_selector}|>

<|part|render={selected_representation == "Country"}|
"""+final_country_md+"""
|>

<|part|render={selected_representation == "Map"}|
"""+map_md+"""
|>
""" 

In [40]:
gui.stop()
page.set_content(main_page)
gui.run()

Gui server has been stopped
 * Server starting on http://127.0.0.1:5000


![](img/part_render.png)

# Taipy Core
Here are the functions that we are going to use to predict the number of Deaths for a country.
We will:
- preprocess the data (_preprocess_),
- create a training and testing database (_make_train_test_data_),
- train a model (_train_model_),
- generate predictions (_forecast_),
- generate a dataframe with the historical data and the predictions (_result_)

![](img/all_architecture.svg)

In [41]:
def preprocess(initial_data, country):
    data = initial_data.groupby(["Country/Region",'Date'])\
                       .sum()\
                       .dropna()\
                       .reset_index()

    preprocess_data = data.loc[data['Country/Region']==country].reset_index(drop=True)
    return preprocess_data
    
    
def make_train_test_data(preprocess_data):
    perc_test = 0.3
    nb_to_predict = int(perc_test*len(preprocess_data))
    train_data = preprocess_data[:-nb_to_predict]
    return train_data, nb_to_predict


def train_model(train_data):
    model = auto_arima(train_data['Deaths'],
                       start_p=1, start_q=1,
                       max_p=5, max_q=5,
                       start_P=0, seasonal=False,
                       d=1, D=1, trace=True,
                       error_action='ignore',  
                       suppress_warnings=True)
    model.fit(train_data['Deaths'])
    return model


def forecast(model, nb_to_predict):
    predictions = model.predict(n_periods=nb_to_predict)
    return predictions


def result(preprocess_data, predictions, nb_to_predict):
    pred_series = pd.Series([np.NaN]*(len(preprocess_data)-nb_to_predict) + list(predictions), name='Predictions')
    return pd.concat([preprocess_data, pred_series], axis=1)

First we must define the Data Nodes then the tasks (associated to the Python function). Furthermore, we gather these tasks into different pipelines and these pipelines into a scenario.

A Data Nod needs a **unique id**. If needed, the storage type can be changed for CSV and SQL. Other parameters are then needed.

### Data Nodes and Task for preprocess

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [42]:
initial_data_cfg = Config.configure_data_node(id="initial_data",
                                              storage_type="csv",
                                              path=path_to_data,
                                              cacheable=True,
                                              validity_period=dt.timedelta(days=5))

country_cfg = Config.configure_data_node(id="country", default_data="France",
                                         cacheable=True, validity_period=dt.timedelta(days=5))

preprocess_data_cfg =  Config.configure_data_node(id="preprocess_data",
                                                  cacheable=True, validity_period=dt.timedelta(days=5))


task_preprocess_cfg = Config.configure_task(id="task_preprocess_data",
                                           function=preprocess,
                                           input=[initial_data_cfg, country_cfg],
                                           output=preprocess_data_cfg)

### Data Nodes and Task for make_train_test_data

<img src="img/make_train_test_data.svg" alt="drawing" width="500"/>

In [43]:
train_data_cfg =  Config.configure_data_node(id="train_data", cacheable=True, validity_period=dt.timedelta(days=5))

nb_to_predict_cfg = Config.configure_data_node(id="nb_to_predict",
                                               cacheable=True, validity_period=dt.timedelta(days=5))

task_train_test_cfg = Config.configure_task(id="task_make_train_test_data",
                                           function=make_train_test_data,
                                           input=preprocess_data_cfg,
                                           output=[train_data_cfg, nb_to_predict_cfg])

### Data Nodes and Task for train_model

<img src="img/train_model.svg" alt="drawing" width="500"/>

In [44]:
model_cfg = Config.configure_data_node(id="model", cacheable=True, validity_period=dt.timedelta(days=5))

task_train_cfg = Config.configure_task(id="task_train",
                                      function=train_model,
                                      input=train_data_cfg,
                                      output=model_cfg)

### Data Nodes and Task for forecast

<img src="img/forecast.svg" alt="drawing" width="500"/>

In [48]:
# 1) configure the predictions Data Node (predictions_cfg)

# 2) configure the task_forecast task (task_forecast_cfg)
# (input=[model, nb_to_predict] / output=predictions_cfg / function=forecast)

SyntaxError: invalid syntax (Temp/ipykernel_15988/1731635280.py, line 4)

Traceback (most recent call last):
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 999, in handle_one_response
    self.run_application()
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 945, in run_application
    self.result = self.application(self.environ, self.start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask\app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask_socketio\__init__.py", line 43, in __call__
    return super(_SocketIOMiddleware, self).__call__(environ,
  File "C:\Users\ohbok\anaconda3\lib\site-packages\engineio\middleware.py", line 63, in __call__
    return self.engineio_app.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\socketio\server.py", line 597, in handle_request
    return self.eio.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\s

Traceback (most recent call last):
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 999, in handle_one_response
    self.run_application()
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 945, in run_application
    self.result = self.application(self.environ, self.start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask\app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask_socketio\__init__.py", line 43, in __call__
    return super(_SocketIOMiddleware, self).__call__(environ,
  File "C:\Users\ohbok\anaconda3\lib\site-packages\engineio\middleware.py", line 63, in __call__
    return self.engineio_app.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\socketio\server.py", line 597, in handle_request
    return self.eio.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\s

### Data Nodes and Task for result

<img src="img/result.svg" alt="drawing" width="500"/>

In [46]:
# 1) configure the result Data Node (result_cfg)
# 2) configure the task_result task (task_result_cfg)
# (input=[preprocess_data, predictions, nb_to_predict] / output=result_cfg / function=result)

## [Configuration of pipelines](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_pipeline)

In [47]:
pipeline_preprocessing_cfg = Config.configure_pipeline(id="pipeline_preprocessing",
                                                       task_configs=[task_preprocess_cfg, task_train_test_cfg])

pipeline_train_cfg = Config.configure_pipeline(id="pipeline_train",
                                               task_configs=[task_train_cfg])

pipeline_forecast_cfg = Config.configure_pipeline(id="pipeline_forecast",
                                                       task_configs=[task_forecast_cfg, task_result_cfg])

NameError: name 'task_forecast_cfg' is not defined

## [Configuration of scénario](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_scenario)

In [ ]:
scenario_cfg = Config.configure_scenario(id='scenario', pipeline_configs=[pipeline_preprocessing_cfg,
                                                                         pipeline_train_cfg,
                                                                         pipeline_forecast_cfg])

## Creation and submit of scenario

In [ ]:
scenario = tp.create_scenario(scenario_cfg, name='First Scenario')
tp.submit(scenario)

In [ ]:
scenario.model.read()

In [ ]:
scenario.result.read()

In [ ]:
scenario.predictions.read()

## Caching
Some job are skipped because no change has been done to the "input" Data Nodes.

In [ ]:
tp.submit(scenario)

## Write in data nodes

To write a data node:

`<Data Node>.write(new_value)`

In [ ]:
scenario.country.write('US')
tp.submit(scenario)
scenario.result.read()

## Simple framework

In [ ]:
scenario = tp.create_scenario(scenario_cfg, name='Second Scenario')
tp.submit(scenario)

In [ ]:
[s.model.read() for s in tp.get_scenarios()]

## Create a Gui for the backend
_scenario_selector_ lets you choose a scenario and display its results.

In [ ]:
scenario_selector = [(s.id, s.name) for s in tp.get_scenarios()]
selected_scenario = scenario.id
print(scenario_selector,'\n', selected_scenario)

In [ ]:
result = scenario.result.read()
result

**Tips** : the _value_by_id_ property if set to True for a selected will make _selected_scenario_ directly refer to the first element of the tupple (here the id)

![](img/predictions.png)

In [ ]:
prediction_md = """
<|layout|columns=1 5 1 3|
<|{scenario_name}|input|label=Name|>

<br/>
<|Create|button|on_action=create_new_scenario|>

<|{selected_country}|selector|lov={selector_country}|dropdown|on_change=on_change_country|label=Country|>

<br/>
<|Submit|button|on_action=submit_scenario|>

<|{selected_scenario}|selector|lov={scenario_selector}|on_change=actualize_graph|dropdown|value_by_id|label=Scenario|>
|>

<|{result}|chart|x=Date|y[1]=Deaths|type[1]=bar|y[2]=Predictions|>
"""

In [ ]:
def create_new_scenario(state):
    scenario = tp.create_scenario(scenario_cfg, name=state.scenario_name)
    state.scenario_selector += [(scenario.id, scenario.name)]

In [ ]:
def actualize_graph(state):
    scenario = tp.get(state.selected_scenario)
    # 1) update the result dataframe
    # 2) change selected_country with the predicted country of the scenario

In [ ]:
def submit_scenario(state):
    # 1) get the selected scenario
    # 2) write in country Data Node, the selected country
    # 3) submit the scenario
    # 4) actualize le graph avec actualize_graph
    ...

In [ ]:
gui.stop()
page.set_content(prediction_md)
gui.run()

# Multi-pages and Taipy Rest

To create a multi-pages app, we only need a dictionary with names as the keys and the Markdowns as the values.

The _navbar_ control (<|navbar|>) has a default behaviour. It redirects to the different pages of the app automatically. Other solutions exists.

![](img/multi_pages.png)

In [ ]:
navbar_md = "<center>\n<|navbar|>\n</center>"

pages = {
    "Map":navbar_md+map_md,
    "Country":navbar_md+final_country_md,
    "Predictions":navbar_md+prediction_md
}

rest = tp.Rest()

gui_multi_pages = Gui(pages=pages)
tp.run(gui_multi_pages, rest, dark_mode=False, port=5006)

The WebSocket transport is not available, you must install a WebSocket server that is compatible with your async mode to enable it. See the documentation for details. (further occurrences of this error will be logged with level INFO)
Invalid session qHOmYp0SabKwaX4lAAAC (further occurrences of this error will be logged with level INFO)
Traceback (most recent call last):
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 999, in handle_one_response
    self.run_application()
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 945, in run_application
    self.result = self.application(self.environ, self.start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask\app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask_socketio\__init__.py", line 43, in __call__
    return super(_SocketIOMiddleware, self).__call__(environ,
  File "C:\Users\ohbok\ana

Traceback (most recent call last):
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 999, in handle_one_response
    self.run_application()
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 945, in run_application
    self.result = self.application(self.environ, self.start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask\app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask_socketio\__init__.py", line 43, in __call__
    return super(_SocketIOMiddleware, self).__call__(environ,
  File "C:\Users\ohbok\anaconda3\lib\site-packages\engineio\middleware.py", line 63, in __call__
    return self.engineio_app.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\socketio\server.py", line 597, in handle_request
    return self.eio.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\s

Traceback (most recent call last):
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 999, in handle_one_response
    self.run_application()
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 945, in run_application
    self.result = self.application(self.environ, self.start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask\app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask_socketio\__init__.py", line 43, in __call__
    return super(_SocketIOMiddleware, self).__call__(environ,
  File "C:\Users\ohbok\anaconda3\lib\site-packages\engineio\middleware.py", line 63, in __call__
    return self.engineio_app.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\socketio\server.py", line 597, in handle_request
    return self.eio.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\s

Traceback (most recent call last):
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 999, in handle_one_response
    self.run_application()
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 945, in run_application
    self.result = self.application(self.environ, self.start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask\app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask_socketio\__init__.py", line 43, in __call__
    return super(_SocketIOMiddleware, self).__call__(environ,
  File "C:\Users\ohbok\anaconda3\lib\site-packages\engineio\middleware.py", line 63, in __call__
    return self.engineio_app.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\socketio\server.py", line 597, in handle_request
    return self.eio.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\s

Traceback (most recent call last):
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 999, in handle_one_response
    self.run_application()
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 945, in run_application
    self.result = self.application(self.environ, self.start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask\app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask_socketio\__init__.py", line 43, in __call__
    return super(_SocketIOMiddleware, self).__call__(environ,
  File "C:\Users\ohbok\anaconda3\lib\site-packages\engineio\middleware.py", line 63, in __call__
    return self.engineio_app.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\socketio\server.py", line 597, in handle_request
    return self.eio.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\s

Traceback (most recent call last):
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 999, in handle_one_response
    self.run_application()
  File "C:\Users\ohbok\anaconda3\lib\site-packages\gevent\pywsgi.py", line 945, in run_application
    self.result = self.application(self.environ, self.start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask\app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\flask_socketio\__init__.py", line 43, in __call__
    return super(_SocketIOMiddleware, self).__call__(environ,
  File "C:\Users\ohbok\anaconda3\lib\site-packages\engineio\middleware.py", line 63, in __call__
    return self.engineio_app.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\site-packages\socketio\server.py", line 597, in handle_request
    return self.eio.handle_request(environ, start_response)
  File "C:\Users\ohbok\anaconda3\lib\s